# 3.2: Hierarchical clustering

### 3.2.1 Show a low level dendrogram containing 5 instances of each digit ( one person ).


### 3.2.2 Use K-Means clustering to compress each digit into 5 clusters, as done in 3.1.1, and perform hierarchical clustering to show a low level dendrogram of this (one person).


### 3.1.3 Discuss the results and relate them to the cross validation tables from k-NN classification